In [23]:
import pandas as pd 

In [24]:
def process_data(file_path):
    df = pd.read_csv(file_path)
    months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sept', 'Oct', 'Nov', 'Dec']

    columns_to_keep = ['resources.description', 'gross_amount', 'gocardless_fees', 'app_fees', 'net_amount', 'payouts.arrival_date', 'payments.metadata.Member', 'payments.metadata.References']
    df = df[columns_to_keep]
    df['year'] = df['resources.description'].str.extract(r'(\d{4})')

    # Initialize an empty list to store subsets
    dfs = []
    

    # Iterate over activity types
    for activity_type in ['Activities', 'Subscription']:
        subset = df[df['resources.description'].str.contains(activity_type, case=False, na=False)].copy()
        subset['schedule'] = activity_type

        if activity_type == 'Subscription':
            # Perform additional transformations for 'Subscription'
            subset['resources.description'] = subset['resources.description'].str.replace('[()2023-2024]', '')
            subset['resources.description'] = subset['resources.description'].str.split()
            subset['possible_month1'] = subset['resources.description'].apply(lambda x: x[-1] if x else None)
            subset['possible_month2'] = subset['resources.description'].apply(lambda x: x[-2] if len(x) >= 2 else None)

            # Filter rows with valid months
            subset = subset[subset['possible_month1'].isin(months) | subset['possible_month2'].isin(months)].copy()

    dfs.append(subset)
            
    # Concatenate the subsets
    df = pd.concat(dfs, ignore_index=True)



    return df

# Example usage
file_path = 'inputs2/payout_transactions_reconciliation-export-EX00036AK3273D.csv'
df = process_data(file_path)
df


,resources.description,gross_amount,gocardless_fees,app_fees,net_amount,payouts.arrival_date,payments.metadata.Member,payments.metadata.References,year,schedule,possible_month1,possible_month2


In [25]:
subset = df.copy()

subset['resources.description'] = subset['resources.description'].str.replace(')', '')
subset['resources.description'] = subset['resources.description'].str.replace('(', '')
subset['resources.description'] = subset['resources.description'].str.replace('2024', '')
subset['resources.description'] = subset['resources.description'].str.replace('2023', '')

subset['resources.description'] = subset['resources.description'].str.split(' ', expand=False)
subset['possible_month1'] = subset['resources.description'].apply(lambda x: x[-1] if x else None)
subset['possible_month2'] = subset['resources.description'].apply(lambda x: x[-2] if x else None)




subset.sample(20)


ValueError: a must be greater than 0 unless no samples are taken